# Radiology Work Allocation — Agent Walkthrough

This notebook acts as a narrated “movie script” of the five-agent pipeline. Each scenario shows every agent’s output in plain English, along with a transparent scoring table and the final LLM explanation.


## Notebook structure

1. **Environment preparation** — configure SQLite, seed the schema + CSV fixtures.
2. **Helper utilities** — reusable functions to run each agent step-by-step and print human-friendly logs.
3. **Scenario scripts** — three complete runs (Priority 5, 2, 3), each showing:
   - Agent 1 → Agent 5 outputs (one block per agent)
   - A scoring table for Agent 4
   - Final assignment summary + LLM explanation


In [ ]:
import csv
import importlib
import os
import sqlite3
from pathlib import Path

import pandas as pd
from IPython.display import display, Markdown

REPO_ROOT = Path.cwd()
if not (REPO_ROOT / "notebooks").exists():
    REPO_ROOT = REPO_ROOT.parent

DB_PATH = REPO_ROOT / "notebooks" / "demo_work_allocation.db"
os.environ["DB_DIALECT"] = "sqlite"
os.environ["SQLITE_PATH"] = str(DB_PATH)
INFRA_DIR = REPO_ROOT / "infra" / "mysql_init"
DB_PATH.unlink(missing_ok=True)

conn = sqlite3.connect(DB_PATH)
conn.executescript(
    """
    CREATE TABLE resources (
        resource_id TEXT PRIMARY KEY,
        name TEXT,
        specialty TEXT,
        skill_level INTEGER,
        total_cases_handled INTEGER
    );
    CREATE TABLE resource_calendar (
        calendar_id TEXT PRIMARY KEY,
        resource_id TEXT,
        date TEXT,
        available_from TEXT,
        available_to TEXT,
        current_workload INTEGER
    );
    CREATE TABLE specialty_mapping (
        work_type TEXT PRIMARY KEY,
        required_specialty TEXT,
        alternate_specialty TEXT
    );
    CREATE TABLE work_requests (
        work_id TEXT PRIMARY KEY,
        work_type TEXT,
        description TEXT,
        priority INTEGER,
        timestamp TEXT,
        status TEXT,
        assigned_to TEXT
    );
    """
)


def insert_from_csv(name, table):
    with open(INFRA_DIR / f"{name}.csv", newline="", encoding="utf-8") as handle:
        reader = csv.DictReader(handle)
        rows = list(reader)
    columns = reader.fieldnames
    placeholders = ",".join(["?"] * len(columns))
    values = [tuple(r.get(col) or None for col in columns) for r in rows]
    conn.executemany(
        f"INSERT INTO {table} ({','.join(columns)}) VALUES ({placeholders})",
        values,
    )


datasets = ["resources", "resource_calendar", "specialty_mapping", "work_requests"]
for dataset in datasets:
    insert_from_csv(dataset, dataset)

conn.commit()
conn.close()

display(Markdown(f"**SQLite workspace ready:** `{DB_PATH}` seeded with {len(datasets)} tables."))

# Reload config + DB connection so the agents pick up the local SQLite DSN
from services.api.app import config as config_module
from services.api.app.db import mysql as mysql_module
from services.api.app.db import repositories as repo_module

importlib.reload(config_module)
importlib.reload(mysql_module)
importlib.reload(repo_module)



**SQLite workspace ready:** `c:\GenAI\work-allocation\notebooks\demo_work_allocation.db` seeded with 4 tables.

ModuleNotFoundError: No module named 'services'

## Agent plumbing & helper utilities


In [ ]:
from services.api.app.controllers.assignment_controller import AssignmentController
from services.api.app.agents.add_work_agent import AddWorkAgent
from services.api.app.agents.work_analyzer_agent import WorkAnalyzerAgent
from services.api.app.agents.resource_finder_agent import ResourceFinderAgent
from services.api.app.agents.availability_checker_agent import AvailabilityCheckerAgent
from services.api.app.agents.assignment_agent import AssignmentAgent

controller = AssignmentController()
add_agent: AddWorkAgent = controller.add_agent
analyzer: WorkAnalyzerAgent = controller.analyzer
finder: ResourceFinderAgent = controller.finder
checker: AvailabilityCheckerAgent = controller.checker
assigner: AssignmentAgent = controller.assigner

print("Controller ready. Agents:")
for agent_name in [
    add_agent.__class__.__name__,
    analyzer.__class__.__name__,
    finder.__class__.__name__,
    checker.__class__.__name__,
    assigner.__class__.__name__,
]:
    print(f" • {agent_name}")


### Logging helpers & scoring table renderer


In [ ]:
from IPython.display import Markdown
from datetime import date, time


def headline(text: str):
    print(f"\n{'=' * 100}\n{text}\n{'=' * 100}")


def log_block(title: str, lines):
    print(f"\n{title}\n{'-' * len(title)}")
    for line in lines:
        print(f"✓ {line}")


def render_scoring_table(scored_candidates):
    if not scored_candidates:
        display(Markdown("_No candidates were scored for this request._"))
        return
    rows = []
    for c in scored_candidates:
        breakdown = c.get("breakdown", {})
        rows.append(
            {
                "Resource": f"{c.get('resource_id')} — {c.get('name', 'N/A')}",
                "Role match": f"{breakdown.get('role', 0):.2f}",
                "Skill": f"{breakdown.get('skill', 0):.2f}",
                "Experience": f"{breakdown.get('experience', 0):.2f}",
                "Availability": f"{breakdown.get('availability', 0):.2f}",
                "Workload": f"{breakdown.get('workload', 0):.2f}",
                "Priority bonus": f"{breakdown.get('priority_bonus', 0):.2f}",
                "Final score": f"{c.get('score', 0):.3f}",
            }
        )
    display(pd.DataFrame(rows))


def run_scenario(
    label,
    work_type,
    description,
    priority,
    scheduled_date,
    scheduled_time,
    llm_provider="template",
):
    headline(label)
    payload = {
        "work_type": work_type,
        "description": description,
        "priority": priority,
        "scheduled_date": scheduled_date,
        "scheduled_time": scheduled_time,
    }

    a1 = add_agent.run(payload)
    log_block(
        "Agent 1 — AddWorkAgent",
        [
            f"Work {a1['work_id']} created with priority {a1['priority']} ({a1['work_type']}).",
            f"Scheduled for {scheduled_date} @ {scheduled_time}.",
        ],
    )

    a2 = analyzer.run({"work_id": a1["work_id"]})
    log_block(
        "Agent 2 — WorkAnalyzerAgent",
        [
            f"Required specialty determined: {a2['required_specialty']}",
            f"Backup specialty allowed: {a2['alternate_specialty']}",
        ],
    )

    a3 = finder.run(a2)
    candidate_ids = [c["resource_id"] for c in a3["candidates"]]
    matches_line = (
        f"{len(candidate_ids)} matching radiologists found → {', '.join(candidate_ids)}"
        if candidate_ids
        else "No direct specialty matches; expanded search to entire roster."
    )
    log_block("Agent 3 — ResourceFinderAgent", [matches_line])

    a4 = checker.run(a3)
    top_candidate = a4["scored_candidates"][0] if a4["scored_candidates"] else None
    best_line = (
        f"Best candidate: {top_candidate['resource_id']} ({top_candidate['name']}) with score {top_candidate['score']:.3f}"
        if top_candidate
        else "No suitable candidates scored."
    )
    log_block(
        "Agent 4 — AvailabilityCheckerAgent",
        [
            f"Evaluated {len(a4['scored_candidates'])} candidate(s).",
            best_line,
        ],
    )
    render_scoring_table(a4["scored_candidates"])

    assignment_input = {
        **a4,
        "work_type": a2["work_type"],
        "priority": a2["priority"],
        "scheduled_timestamp": a2["scheduled_timestamp"],
    }
    a5 = assigner.run(assignment_input, llm_provider=llm_provider)
    log_block(
        "Agent 5 — AssignmentAgent",
        [
            f"Work {a5['work_id']} assigned to {a5['assigned_to']} ({a5['selected'].get('name')}).",
            f"LLM Explanation: {a5['explanation']}",
        ],
    )

    display(
        Markdown(
            f"""
**Final Assignment Summary**  
• **Work ID:** `{a5['work_id']}` • `{a5['work_type']}` (priority {a5['priority']})  
• **Scheduled:** {scheduled_date} @ {scheduled_time}  
• **Assigned To:** **{a5['selected'].get('name')}** ({a5['selected'].get('specialty')})  
• **Why:** {a5['explanation']}
            """
        )
    )

    return {
        "work_id": a5["work_id"],
        "assignment": a5,
        "analysis": a2,
        "scored": a4,
    }


## Scenario 1 — Priority 5 (Urgent MRI brain)

> *Emergency stroke protocol requiring immediate neurological expertise and rapid turnaround.*


In [ ]:
scenario_1 = run_scenario(
    label="Scenario 1 — Priority 5 (Urgent MRI_Brain)",
    work_type="MRI_Brain",
    description="Acute stroke evaluation with neurologic deficits; requires immediate attention.",
    priority=5,
    scheduled_date="2024-11-12",
    scheduled_time="09:00",
    llm_provider="template",
)


## Scenario 2 — Priority 2 (Routine abdominal ultrasound)

> *Non-urgent kidney stone follow-up requiring steady workload management and balanced scheduling.*


In [ ]:
scenario_2 = run_scenario(
    label="Scenario 2 — Priority 2 (Routine Ultrasound_Abdomen)",
    work_type="Ultrasound_Abdomen",
    description="Routine renal ultrasound to monitor stone progression; no immediate urgency.",
    priority=2,
    scheduled_date="2024-11-11",
    scheduled_time="11:30",
    llm_provider="template",
)


## Scenario 3 — Priority 3 (Specialized musculoskeletal study)

> *Orthopedic follow-up where musculoskeletal expertise beats general radiology.*


In [ ]:
scenario_3 = run_scenario(
    label="Scenario 3 — Priority 3 (Specialized X_Ray_Bone)",
    work_type="X_Ray_Bone",
    description="High-complexity joint instability evaluation requiring MSK specialization.",
    priority=3,
    scheduled_date="2024-11-10",
    scheduled_time="14:15",
    llm_provider="template",
)


## Cross-scenario assignment snapshot


In [ ]:
summary_rows = []
for label, scenario in [
    ("Priority 5 (Urgent)", scenario_1),
    ("Priority 2 (Routine)", scenario_2),
    ("Priority 3 (Specialized)", scenario_3),
]:
    assignment = scenario["assignment"]
    selected = assignment["selected"]
    summary_rows.append(
        {
            "Scenario": label,
            "Work Type": scenario["analysis"]["work_type"],
            "Priority": assignment["priority"],
            "Assigned To": selected["name"],
            "Specialty": selected.get("specialty"),
            "Final Score": f"{selected.get('score', 0):.3f}",
            "LLM Explanation": assignment["explanation"],
        }
    )

display(pd.DataFrame(summary_rows))


In [ ]:
# Legacy HTTP smoke-test cells removed in favor of the narrated agent walkthrough above.
